## 1. 多头注意力机制

**为什么多头注意力（Multi-Head Attention）有效？**

多头注意力通过并行计算多个独立的注意力头，让模型从不同角度（如局部依赖、长程依赖、语法结构等）学习输入序列的多样化特征，从而提升模型的表达能力和泛化能力。

In [ ]:

import torch
import torch.nn as nn 


class MHA(nn.Module):
    def __init__(self, hidden_dim=5, head=2,**kwargs):
        super().__init__()
        self.qkv_proj = nn.Linear(hidden_dim, 3 * hidden_dim)  # 合并QKV投影以提高效率
        self.hidden_dim=hidden_dim
        self.head=head

    def softmax(self,x):
        x_max=torch.max(x,dim=-1,keepdim=True).values
        x_exp=torch.exp(x-x_max)
        x_exp_sum=torch.sum(x_exp,dim=-1,keepdim=True)
        
        return x_exp/x_exp_sum

    def forward(self,x):
        bs,l,dim=x.shape
        assert dim ==self.hidden_dim
        qkv=self.qkv_proj(x)
        q, k, v = qkv.chunk(3, dim=-1)  # 正确方式
        # q,k,v=qkv.unbind(2)
        dim0=dim//self.head
        q=q.reshape(bs,l,self.head,dim0).transpose(1,2)
        k=k.reshape(bs,l,self.head,dim0).transpose(1,2)
        v=v.reshape(bs,l,self.head,dim0).transpose(1,2)

        qk=torch.matmul(q,k.transpose(2,3))/(dim0**0.5)
        # qk=qk/torch.sqrt(dim0)

        qk=self.softmax(qk)

        atten=torch.matmul(qk,v)    
        return atten.transpose(1,2).reshape(bs,l,dim)
        
        
        


x=torch.randn(4,10,6)  
mha=MHA(hidden_dim=6,head=2)
out=mha(x)

torch.Size([4, 10, 18])
18


## 2. 稀疏注意力（Sparse Attention）在保持模型性能的同时，通过减少计算量来提高效率，其效果不会显著下降的原因可以归结为以下几点？

1. 在标准注意力中，大部分位置的注意力权重接近于零，只有少数位置对当前token有显著贡献。
2. 稀疏注意力可视为对稠密注意力矩阵的低秩近似（保留主要特征，忽略次要特征），理论证明这种近似在多数任务中误差可控。

## 3. 局部窗口类稀疏机制

每个 Query 仅与前后 k 个相邻元素计算注意力

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

class EfficientBandMHA(nn.Module):
    def __init__(self, hidden_dim=5, head=4, band_width=2, **kwargs):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.head = head
        self.head_dim = hidden_dim // head
        self.band_width = band_width  # 每个位置仅关注前后band_width个位置
        
        # 投影层
        self.qkv_proj = nn.Linear(hidden_dim, 3 * hidden_dim)  # 合并QKV投影以提高效率
        self.out_proj = nn.Linear(hidden_dim, hidden_dim)
        
        assert hidden_dim % head == 0, "hidden_dim must be divisible by head"

    def forward(self, x):
        bs, seq_len, dim = x.shape
        
        # 一次性计算QKV并拆分 (优化内存访问)
        qkv = self.qkv_proj(x).reshape(bs, seq_len, 3, self.head, self.head_dim)
        q, k, v = qkv.unbind(2)  # 拆分出Q, K, V，形状均为 (bs, seq_len, head, head_dim)
        
        # 转置为 (bs, head, seq_len, head_dim)
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        
        # 计算带状注意力的关键：只生成有效区域的注意力分数
        # 1. 确定每个位置i的有效注意力范围
        # 2. 仅计算这些位置的Q-K乘积，避免完整的n×n矩阵
        
        # 存储每个头的注意力结果
        attn_outputs = []
        
        for h in range(self.head):
            # 当前头的Q, K, V
            q_h = q[:, h]  # (bs, seq_len, head_dim)
            k_h = k[:, h]  # (bs, seq_len, head_dim)
            v_h = v[:, h]  # (bs, seq_len, head_dim)
            
            # 存储当前头的注意力分数
            sparse_attn_scores = []
            
            for i in range(seq_len):
                # 计算位置i的有效范围 [start, end)
                start = max(0, i - self.band_width)
                end = min(seq_len, i + self.band_width + 1)
                band_len = end - start
                
                # 仅计算Q[i]与K[start:end]的乘积 (节约计算)
                q_i = q_h[:, i:i+1, :]  # (bs, 1, head_dim)
                k_band = k_h[:, start:end, :]  # (bs, band_len, head_dim)
                
                # 计算注意力分数 (bs, 1, band_len)
                scores = torch.matmul(q_i, k_band.transpose(-2, -1)) / (self.head_dim ** 0.5)
                
                # 对带状区域内的分数做softmax
                scores_softmax = F.softmax(scores, dim=-1)  # (bs, 1, band_len)
                
                # 计算注意力加权和 (bs, 1, head_dim)
                v_band = v_h[:, start:end, :]  # (bs, band_len, head_dim)
                attn_i = torch.matmul(scores_softmax, v_band)  # (bs, 1, head_dim)
                
                sparse_attn_scores.append(attn_i)
            
            # 拼接当前头的所有位置结果 (bs, seq_len, head_dim)
            attn_head = torch.cat(sparse_attn_scores, dim=1)
            attn_outputs.append(attn_head)
        
        # 合并所有头 (bs, seq_len, hidden_dim)
        attn_output = torch.cat(attn_outputs, dim=-1)
        output = self.out_proj(attn_output)
        
        return output


# 测试显存占用对比
if __name__ == "__main__":
    # 生成较长序列以观察显存差异
    bs, seq_len, hidden_dim = 4, 1024, 512
    x = torch.randn(bs, seq_len, hidden_dim)  # 使用GPU测试显存
        
    # 高效带状注意力
    band_width=3
    efficient_band_mha = EfficientBandMHA(
        hidden_dim=hidden_dim, 
        head=8, 
        band_width=band_width
    )
    
    # 测试前向传播
    with torch.no_grad():
        out = efficient_band_mha(x)
        print(f"Input shape: {x.shape}")
        print(f"Output shape: {out.shape}")
        print(f"理论显存节省比例: {1 - (2*band_width + 1)/seq_len:.2%}")
    

Input shape: torch.Size([4, 1024, 512])
Output shape: torch.Size([4, 1024, 512])
理论显存节省比例: 99.32%
